# Salary, Gameplay, and Win Percentage: A Case Study of Three Teams
### Ward Anderson, Ben Brown, Jeff Stevens
### Dr. Kretchmar
### 10/22/18

## Introduction

{intro goes here}

In [9]:
import pandas as pd
import sqlalchemy as sa
import matplotlib.pyplot as plt
import json

#Include inflation data
inflation = pd.read_csv("conversionfactors.csv")

To eliminate the effect of inflation on our salary time series data, we incorporated inflation adjustment data, courtesy of the Oregon State University Political Science Department (see: https://liberalarts.oregonstate.edu/spp/polisci/research/inflation-conversion-factors-convert-dollars-1774-estimated-2024-dollars-recent-year).

In [16]:
def getCreds(filename,subset,defaults={}):
    '''
    This function helps us connect to a database on hadoop2.  The
    userid password information is stored in a local file encoded
    in json format.  
    Inputs:
        filename is where json encoded userid/password information stored
        subset : dictionary key in json file
        defaults : alternative way to connect to database
    Returns subdictionary based on subset parameter.
    '''
    try:
        with open(filename,'r') as file:
            D = json.load(file)
            file.close()
            if D[subset]:
                return D[subset]
            else:
                return defaults
    except:
        return defaults
     
def db_setup(database):
    template = 'mysql+mysqlconnector://{}:{}@hadoop2.mathsci.denison.edu/{}'
    cstring = template.format(creds['user'],creds['password'],database)
    return cstring

creds = getCreds('creds.json','mysql',defaults = {'user':'steven_j1', 'password':'steven_j1'})

cstring = db_setup("lahman2016")
engine = sa.create_engine(cstring)
connection = engine.connect()

InterfaceError: (mysql.connector.errors.InterfaceError) 2013: Lost connection to MySQL server during query (Background on this error at: http://sqlalche.me/e/rvf5)

In [11]:
     #-#-#-#-#-#-#-#-#-#-#-#-#-#-#
########## CLEVELAND INDIANS ##########
     #-#-#-#-#-#-#-#-#-#-#-#-#-#-#

query_main = '''SELECT yearID, teamID, franchID, H/AB AS BA, ERA , W/(W+L) AS Winpct, FP
FROM Teams
WHERE franchID = "CLE";'''

getclesal = "SELECT yearID, AVG(salary) AS CleSalary \
FROM Salaries \
WHERE teamID = 'CLE' \
GROUP BY yearID \
ORDER BY yearID;"

In [12]:
#Converting Cleveland Indians query to DataFrame
result = connection.execute(query_main)
rows = result.fetchall()
df = pd.DataFrame(rows,columns = result.keys())
df.columns = ["Year","Team","Franchise","Batting Average","Earned Run Average", "Win Percentage","Fielding Percentage"]
df = df.drop(columns = ["Team","Franchise"], axis = 1)
df.to_csv('CLE.csv')

NameError: name 'connection' is not defined

In [ ]:
print("Statistics for The Cleveland Indians Baseball Team since 1901")
print(df)

In [ ]:
     #-#-#-#-#-#-#-#-#-#-#-#-#-#-#
########## WASHINGTON NATIONALS ##########
     #-#-#-#-#-#-#-#-#-#-#-#-#-#-#
    
    
WAS_query = '''SELECT t.yearID, t.teamID, t.franchID, t.H, t.AB, t.W, t.L, t.ERA, t.H/t.AB AS BA, t.W/(t.W+t.L) AS Winpct, t.FP
FROM Teams AS t
WHERE t.franchID = "WSN";'''
resultproxy = connection.execute(WAS_query)

getwsnsal = "SELECT yearID, AVG(salary) AS WsnSalary \
FROM Salaries \
WHERE teamID = 'WSN' \
OR teamID = 'WAS' \
OR teamID = 'MON' \
GROUP BY yearID \
ORDER BY yearID;"

In [ ]:
#Main query
tmp = connection.execute(WAS_query)
WAS_stats = pd.DataFrame(tmp.fetchall(), columns = tmp.keys())
print(WAS_stats)

In [ ]:
     #-#-#-#-#-#-#-#-#-#-#-#-#-#-#
########## CINCINNATI REDS ##########
     #-#-#-#-#-#-#-#-#-#-#-#-#-#-#

getredstats = "SELECT t.yearID, t.teamID, t.franchID, t.H, t.AB, t.W, t.L, t.ERA, t.H/t.AB AS BA, t.W/(t.W+t.L) AS Winpct, t.FP \
FROM Teams AS t \
WHERE t.franchID = 'CIN';"

getredsal = "SELECT yearID, AVG(salary) AS RedsSalary \
FROM Salaries \
WHERE teamID = 'CIN' \
GROUP BY yearID \
ORDER BY yearID;"

tmp = connection.execute(getredstats)
redstats = pd.DataFrame(tmp.fetchall(), columns = tmp.keys())

print(redstats)

In [ ]:
#Get salary data
tmp = connection.execute(getredsal)
redsal = pd.DataFrame(tmp.fetchall(), columns = tmp.keys())
tmp = connection.execute(getclesal)
clesal = pd.DataFrame(tmp.fetchall(), columns = tmp.keys())
tmp = connection.execute(getwsnsal)
wsnsal = pd.DataFrame(tmp.fetchall(), columns = tmp.keys())

#Aggregate salary data and inflation data
salary = redsal.merge(clesal, on = "yearID").merge(wsnsal, on = "yearID").merge(inflation, left_on = "yearID", right_on = "Year")
salary = salary.drop("Year", axis = 1)
salary.RedsSalary = pd.to_numeric(salary.RedsSalary)
salary.CleSalary = pd.to_numeric(salary.CleSalary)
salary.WsnSalary = pd.to_numeric(salary.WsnSalary)

#Adjust for inflation
salary.RedsSalary = salary.RedsSalary/salary.Factor
salary.CleSalary = salary.CleSalary/salary.Factor
salary.WsnSalary = salary.WsnSalary/salary.Factor
salary = salary.drop("Factor", axis = 1)

print(salary)

## Analysis

In order to assess the impact of player salaries on win percentage, we plotted win percentage against year, overlaid with information about average salary per team. Salaries were converted to 2017 dollars with the help of the inflation data from Oregon State University. 

In [17]:
def scaleColors(sal):
    '''
    This function takes numeric data (presumably salary data) and returns a list of scaled colors
    representing how close each value falls to the min or max of the input data
    Parameters: a series of numeric data
    Return: a list of color triples
    '''
    colors = []
    mx = max(sal)
    for i in sal:
        colors.append( (1-i/mx, i/mx, 0) )
    return colors

redCol = scaleColors(salary.RedsSalary)
cleCol = scaleColors(salary.CleSalary)
wsnCol = scaleColors(salary.WsnSalary)

plt.scatter(redstats.Winpct[redstats.yearID >= min(salary.yearID)], salary.RedsSalary, c = redCol)
plt.xlabel("Year")
plt.ylabel("Win Percentage")
plt.title("Cincinnati Reds: Win Percentage vs. Year")
plt.show()

plt.scatter(df["Win Percentage"][df.Year >= min(salary.yearID)], salary.CleSalary, c = cleCol)
plt.xlabel("Year")
plt.ylabel("Win Percentage")
plt.title("Cleveland Indians: Win Percentage vs. Year")
plt.show()

plt.scatter(WAS_stats.Winpct[WAS_stats.yearID >= min(salary.yearID)], salary.WsnSalary, c = wsnCol)
plt.xlabel("Year")
plt.ylabel("Win Percentage")
plt.title("Washington Nationals: Win Percentage vs. Year")
plt.show()


NameError: name 'salary' is not defined

Above, green points represent high values of salary, while red points represent low values.